### Marshall, Colorado, USA

**Station Name:** p041 

**Location:** Boulder, CO, USA

**Archive:** [UNAVCO](http://www.unavco.org)

**DOI:**  	[https://doi.org/10.7283/T5R49NQQ](https://doi.org/10.7283/T5R49NQQ)

**Ellipsoidal Coordinates:**

- Latitude: 39.94949

- Longitude: -105.19427

- Height: 1728.842 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/P041)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/P041.sta)

[Google Map Link](https://goo.gl/maps/GwGV8PS4CQVQzYHC7) 

<img src="https://gnss-reflections.org/static/images/P041.jpg" width="500">

In [ ]:
import json
import os
import sys

import ipywidgets as widgets
import pandas as pd
import seaborn as sns;

sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

Fast Mode - this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(value=True, description='Fast Mode', disabled=False, indent=False)
display(weekly)

## Data Summary

The p041 antenna is ~2 meters above the soil surface. It is located at Marshall Mesa, Colorado.
The site is relatively planar and free of obstructions. Since October 2018 the site has 
recorded multi-GNSS signals. Marshall Mesa has been featured in multiple publications on GNSS-IR:

* [Use of GPS Receivers as a Soil Moisture Network for Water Cycle Studies (2008)](https://www.kristinelarson.net/wp-content/uploads/2015/10/larson_soil_grl2008.pdf)

* [Can We Measure Snow Depth with GPS Receivers (2009)](https://www.kristinelarson.net/wp-content/uploads/2015/10/larsonetal_snow_2009.pdf) 

* [Sensing Vegetation Growth with Reflected GPS Signals (2010)](https://www.kristinelarson.net/wp-content/uploads/2015/10/small_etal_2010.pdf) 

### Web App

More information on the station can be obtained from the GNSS-IR Web App, where p041 is one of the example cases for soil moisture and snow depth. [Please note that the app will be analyzing data in real-time, so please wait for the periodogram to appear on the left side of the page. It takes about 5 seconds](https://gnss-reflections.org/fancy6?example=p041). The web app will return a photograph, station coordinates, a Google Earth Map, and a sample periodogram. The periodogram plots the reflector height (RH) in four quadrants (NW, NE, SW, SE), allowing the reflection characteristics and quality of the site to be inferred. For example, variations in topography or buildings blocking the reflections could all affect the periodograms.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=p041" width="1000" height="900"></iframe>

### Setting Azimuth and Elevation Mask

To get a sense of whether an azimuth or elevation mask is appropriate, check the [Reflection Zone Mapping in the web app](https://gnss-reflections.org/rzones?station=p041&lat=39.9495&lon=-105.1943&height=1728.842&msl=on&RH=2&eang=2&azim1=0&azim2=360).  
In the linked page, the reflection zones at 5, 10, and 15-degree elevation angles are plotted as colored ellipses surrounding the station, all overlaid on a Google Earth map.  Higher elevation angles are closer to the station.
As can be seen from the Google Earth map, this site is relatively flat with no major obstacles to interfere with reflected signals.  Therefore the elevation angles can be left at default settings and no azimuth masks are required.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones?station=p041&lat=39.9495&lon=-105.1943&height=1728.842&msl=on&RH=2&eang=2&azim1=0&azim2=360" width="950" height="500"></iframe>

### Make SNR File

Begin by making an SNR file. We'll look at the year 2020 and day of year 132

In [ ]:
station = 'p041'
year = 2021 
doy = 5

lat = 39.9495
long = -105.1943
height = 1728.856

In [ ]:
# Can view the parameters for the rinex2snr function here
rinex2snr?

In [ ]:
rinex2snr(station, year, doy)

### Take a Quick Look at the Data

Then use **quickLook** to analyze the reflection characteristics of the site [(For details on quickLook output)](../../docs/quickLook_desc.md).

The default return is for the L1 frequency:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

Now try looking at the periodogram for L2C:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=20)

Note that there are more colors in the L1 plots than in the L2C plots. That is the result of the fact that there are more L1 satellites than L2C satellites.

Now try L5:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=5)

The L5 signal has only been available on satellites launched after 2010, so there are
fewer satellite tracks than either L1 or L2C.

The *quickLook* code has multiple options. For example, it is possible change the reflector height range:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, h1=0.5, h2=10)

To look at Glonass and Galileo signals, the SNR files must be created using the -orb gnss flag.
If you have already made a file using only the GPS data, you will need the overwrite flag.

In [ ]:
rinex2snr(station, year, doy, orb='gnss', overwrite=True)

In [ ]:
values, metrics = quicklook(station, year, doy=doy, h1=0.5, h2=10, fr=101)

Beidou signals are tracked at this site, but unfortunately the data are not available in the RINEX 2.11 file.
They are very likely available in the RINEX 3 file, so you are encouraged to look there.

**quickLook** is meant to be a visual assessment of the spectral characteristics at a given site on a given day. For routine analysis, one must use  **gnssir**.

## Analyze the Data

We will start by setting up the analysis parameters. These are stored
in a json file. In this case, the p041 RINEX data are multi-gnss, so you could
set the options to allow all frequencies from all constellations by setting the parameter allfreq=True.

We are going to concentrate on GPS-only, which is the default. We have set stricter QC values by
setting the amplitude minimum to 8 and the peak 2 noise ratio to 3.2:

In [ ]:
# Here we can see what parameters are available and what the defaults are:
make_json?

In [ ]:
# we saved the lat, long, and height earlier
make_json(station, lat, long, height, peak2noise=3.2, ampl=8)

Because the site is uniformly flat, the parameters canbe left at default settings. The elevation angles for the SNR data are set to minimum and maximum values of 5 and 25 degrees, respectively.  The json output will be stored in $REFL_CODE/input/p041.json.

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/p041.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)

os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

We are going to look at a subset of p041 data from 2019/2020 to look at changes due to
snow accumulation. The series will begin doy 245 (2019) and end on doy 70 (2020).

In [ ]:
rinex2snr(station, year=2019, doy=245, doy_end=365, archive='unavco', weekly=weekly.value)

In [ ]:
rinex2snr(station, year=2020, doy=1, doy_end=70, archive='unavco', weekly=weekly.value)

Now run gnssir for 2019/2020:

In [ ]:
#gnssir?

In [ ]:
year = 2019
doy = 1
doy_end = 366
year_end = 2020
pt=False
gnssir(station, year, doy, doy_end=doy_end, year_end=year_end, plt=pt, screenstats=False)

Typically a daily average is sufficient for climatology studies.
To ensure the average is meaningful and not impacted by large outliers,
a median filter (meters) is used and a minimum number of tracks is required. Here a median filter of 0.15 meter is used and 80 tracks are required. Either of these parameters can be changed depending on your site.
In this particular example, I only used three GPS frequencies, L1, L2C, and L5.

In [ ]:
#daily_avg?

In [ ]:
daily_avg(station, medfilter=.15, ReqTracks=80, plt=False, txtfile='p041-dailyavg.txt')

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}/{station}_allRH.txt'
data = gnssrefl_helpers.read_rh_files(filepath)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim(2.05, 1.5)
g.set_ylabel('Reflector Height (m)');

A summary of how many values are being used in the daily average:

In [ ]:
plt.figure(figsize=(8,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

And the daily RH average:

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}/{station}-dailyavg.txt'
data = gnssrefl_helpers.read_rh_files(filepath)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(1.98,1.7)
g.set_ylabel('Reflector Height (m)')
g.set_title(f'{station.upper()}: Daily Mean Reflector Height', size=16);